# Milestone 08: 重構需求規格 | Requirements Specification

> **專案名稱**: 專案模組化重構  
> **整合章節**: Ch27-30 (Part VIII: Engineering Practices)  
> **版本**: 1.0.0

## 📋 目錄 | Table of Contents

1. [專案概述](#1-專案概述)
2. [模組化結構設計](#2-模組化結構設計)
3. [檔案組織規範](#3-檔案組織規範)
4. [PEP 8 檢核清單](#4-pep-8-檢核清單)
5. [文件撰寫規範](#5-文件撰寫規範)
6. [Git 工作流程](#6-git-工作流程)
7. [完成檢核清單](#7-完成檢核清單)

---

## 1. 專案概述 | Project Overview

### 1.1 重構目標

選擇 M01-M07 任一專案，將單一檔案的程式碼轉換為符合工程標準的模組化專案。

**重構前 (Before)**:
```
milestone0X-project/
└── solution.ipynb    # 單一檔案，200-400 行
```

**重構後 (After)**:
```
refactored_project/
├── README.md              # 專案說明文件
├── requirements.txt       # 依賴清單
├── .gitignore            # Git 忽略檔案
├── src/                  # 原始碼目錄
│   ├── __init__.py       # 套件初始化
│   ├── main.py           # 主程式入口
│   ├── core.py           # 核心邏輯
│   ├── ui.py             # 使用者界面
│   ├── storage.py        # 資料儲存 (如需要)
│   ├── utils.py          # 工具函式
│   └── exceptions.py     # 自訂例外 (如需要)
├── data/                 # 資料檔案目錄 (如需要)
│   └── *.json / *.csv
└── tests/                # 測試目錄 (選做)
    └── test_*.py
```

### 1.2 核心原則

1. **單一職責原則 (Single Responsibility Principle)**
   - 每個模組只負責一個功能領域
   - 每個函式只做一件事

2. **關注點分離 (Separation of Concerns)**
   - UI 與邏輯分離
   - 資料處理與業務邏輯分離
   - 工具函式獨立

3. **低耦合 (Low Coupling)**
   - 模組間依賴最小化
   - 透過明確的介面溝通

4. **高內聚 (High Cohesion)**
   - 相關功能組織在同一模組
   - 模組內部邏輯緊密相關

---

## 2. 模組化結構設計 | Module Structure Design

### 2.1 核心模組 (core.py)

**職責**：
- 業務邏輯實作
- 資料結構定義
- 演算法實現
- 核心類別定義

**特性**：
- ✅ 純函式設計（Pure Functions）
- ✅ 不依賴使用者輸入輸出
- ✅ 可獨立測試
- ✅ 不包含 `input()` 或 `print()`

**範例**：
```python
# src/core.py
"""核心業務邏輯模組"""

from typing import List, Dict, Optional

class Task:
    """任務類別"""
    
    def __init__(self, title: str, priority: int = 1) -> None:
        self.title: str = title
        self.priority: int = priority
        self.completed: bool = False
    
    def mark_complete(self) -> None:
        """標記任務為完成"""
        self.completed = True


class TaskManager:
    """任務管理器類別"""
    
    def __init__(self) -> None:
        self.tasks: List[Task] = []
    
    def add_task(self, task: Task) -> None:
        """新增任務"""
        self.tasks.append(task)
    
    def get_pending_tasks(self) -> List[Task]:
        """取得未完成的任務"""
        return [t for t in self.tasks if not t.completed]
```

### 2.2 使用者界面模組 (ui.py)

**職責**：
- 顯示資訊給使用者 (`print()`)
- 接收使用者輸入 (`input()`)
- 選單顯示與互動
- 格式化輸出

**特性**：
- ✅ 所有使用者互動集中於此
- ✅ 不包含業務邏輯
- ✅ 呼叫 core 模組的函式

**範例**：
```python
# src/ui.py
"""使用者界面模組"""

from typing import Optional
from .core import Task, TaskManager


def display_menu() -> None:
    """顯示主選單"""
    print("\n=== Todo App ===")
    print("1. 新增任務")
    print("2. 顯示任務")
    print("3. 離開")


def get_menu_choice() -> str:
    """取得使用者選擇"""
    return input("請選擇功能: ").strip()


def display_tasks(manager: TaskManager) -> None:
    """顯示所有任務"""
    tasks = manager.get_pending_tasks()
    if not tasks:
        print("目前沒有任務")
        return
    
    for i, task in enumerate(tasks, 1):
        status = "✓" if task.completed else "☐"
        print(f"{i}. [{status}] {task.title} (優先度: {task.priority})")
```

### 2.3 資料儲存模組 (storage.py)

**職責**：
- 檔案讀寫操作
- JSON/CSV 資料轉換
- 資料序列化與反序列化
- 檔案錯誤處理

**範例**：
```python
# src/storage.py
"""資料儲存模組"""

import json
from pathlib import Path
from typing import List, Dict, Any


def load_json(file_path: Path) -> List[Dict[str, Any]]:
    """從 JSON 檔案載入資料
    
    Args:
        file_path: 檔案路徑
    
    Returns:
        資料列表
    
    Raises:
        FileNotFoundError: 檔案不存在時
        json.JSONDecodeError: JSON 格式錯誤時
    """
    if not file_path.exists():
        return []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)


def save_json(data: List[Dict[str, Any]], file_path: Path) -> None:
    """儲存資料到 JSON 檔案
    
    Args:
        data: 要儲存的資料
        file_path: 檔案路徑
    """
    file_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
```

### 2.4 工具模組 (utils.py)

**職責**：
- 通用輔助函式
- 資料驗證
- 格式化工具
- 常數定義

**範例**：
```python
# src/utils.py
"""工具函式模組"""

from datetime import datetime
from typing import Optional

# 常數定義
DATE_FORMAT = "%Y-%m-%d"
MAX_PRIORITY = 5
MIN_PRIORITY = 1


def validate_priority(priority: int) -> bool:
    """驗證優先度數值
    
    Args:
        priority: 優先度 (1-5)
    
    Returns:
        是否有效
    """
    return MIN_PRIORITY <= priority <= MAX_PRIORITY


def format_date(date: datetime) -> str:
    """格式化日期
    
    Args:
        date: 日期時間物件
    
    Returns:
        格式化的日期字串
    """
    return date.strftime(DATE_FORMAT)


def parse_date(date_str: str) -> Optional[datetime]:
    """解析日期字串
    
    Args:
        date_str: 日期字串 (YYYY-MM-DD)
    
    Returns:
        日期時間物件，解析失敗時回傳 None
    """
    try:
        return datetime.strptime(date_str, DATE_FORMAT)
    except ValueError:
        return None
```

### 2.5 自訂例外模組 (exceptions.py)

**職責**：
- 定義專案特定的例外類別
- 提供清楚的錯誤訊息

**範例**：
```python
# src/exceptions.py
"""自訂例外模組"""


class TaskNotFoundError(Exception):
    """找不到任務時拋出的例外"""
    pass


class InvalidPriorityError(ValueError):
    """優先度數值無效時拋出的例外"""
    
    def __init__(self, priority: int) -> None:
        super().__init__(f"優先度 {priority} 無效，必須介於 1-5 之間")


class InvalidDateFormatError(ValueError):
    """日期格式錯誤時拋出的例外"""
    
    def __init__(self, date_str: str) -> None:
        super().__init__(f"日期格式錯誤: {date_str}，應為 YYYY-MM-DD")
```

### 2.6 主程式模組 (main.py)

**職責**：
- 程式入口點
- 整合各模組功能
- 主流程控制

**範例**：
```python
# src/main.py
"""主程式模組"""

from pathlib import Path
from .core import TaskManager, Task
from .ui import display_menu, get_menu_choice, display_tasks
from .storage import load_json, save_json
from .utils import validate_priority
from .exceptions import InvalidPriorityError


def main() -> None:
    """主程式入口"""
    manager = TaskManager()
    
    # 載入資料
    data_file = Path("data/tasks.json")
    # ... 載入邏輯 ...
    
    # 主迴圈
    while True:
        display_menu()
        choice = get_menu_choice()
        
        if choice == "1":
            # 新增任務
            pass
        elif choice == "2":
            # 顯示任務
            display_tasks(manager)
        elif choice == "3":
            # 儲存並離開
            break
    
    print("再見！")


if __name__ == "__main__":
    main()
```

### 2.7 套件初始化 (\_\_init\_\_.py)

**職責**：
- 定義套件的公開 API
- 簡化匯入路徑

**範例**：
```python
# src/__init__.py
"""Todo App 套件

此套件提供待辦事項管理功能。
"""

__version__ = "1.0.0"
__author__ = "Your Name"

# 定義公開 API
__all__ = [
    "Task",
    "TaskManager",
    "main",
]

from .core import Task, TaskManager
from .main import main
```

---

## 3. 檔案組織規範 | File Organization Standards

### 3.1 目錄結構範本

```
my_project/
├── .git/                     # Git 版本控制目錄
├── .gitignore               # Git 忽略檔案清單
├── README.md                # 專案說明文件
├── requirements.txt         # Python 依賴清單
│
├── src/                     # 原始碼目錄
│   ├── __init__.py          # 套件初始化
│   ├── main.py              # 主程式
│   ├── core.py              # 核心邏輯
│   ├── ui.py                # 使用者界面
│   ├── storage.py           # 資料儲存
│   ├── utils.py             # 工具函式
│   └── exceptions.py        # 自訂例外
│
├── data/                    # 資料檔案目錄
│   ├── .gitkeep             # 保留空目錄
│   └── example.json         # 範例資料
│
└── tests/                   # 測試目錄 (選做)
    ├── __init__.py
    ├── test_core.py
    └── test_utils.py
```

### 3.2 .gitignore 範本

```gitignore
# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python

# 虛擬環境
venv/
env/
ENV/
.venv

# IDE
.vscode/
.idea/
*.swp
*.swo
*~

# 作業系統
.DS_Store
Thumbs.db

# 專案特定
data/*.json
!data/example.json
logs/
*.log

# 測試與覆蓋率
.pytest_cache/
.coverage
htmlcov/
```

### 3.3 requirements.txt 範本

```txt
# 開發工具 (僅開發環境需要)
# flake8==6.0.0
# black==23.3.0
# pytest==7.3.1

# 如果專案沒有外部依賴，可以留空或註解說明
# 本專案僅使用 Python 標準函式庫
```

### 3.4 模組匯入順序

依照 PEP 8 規範，匯入應按以下順序分組，組與組之間空一行：

```python
# 1. 標準函式庫
import json
import sys
from pathlib import Path
from typing import List, Dict, Optional

# 2. 第三方套件
import numpy as np
import pandas as pd

# 3. 本地應用程式/函式庫
from .core import Task, TaskManager
from .utils import validate_priority
```

---

## 4. PEP 8 檢核清單 | PEP 8 Checklist

### 4.1 命名規範

| 類型 | 規範 | 正確範例 | 錯誤範例 |
|:-----|:-----|:---------|:---------|
| 模組名稱 | 小寫，底線分隔 | `task_manager.py` | `TaskManager.py` |
| 套件名稱 | 小寫，不使用底線 | `mypackage` | `my_package` |
| 類別名稱 | 大駝峰 (CapWords) | `TaskManager` | `task_manager` |
| 函式名稱 | 小寫，底線分隔 | `add_task()` | `addTask()` |
| 變數名稱 | 小寫，底線分隔 | `task_list` | `taskList` |
| 常數名稱 | 大寫，底線分隔 | `MAX_PRIORITY` | `maxPriority` |
| 私有成員 | 前綴單底線 | `_internal_var` | `internalVar` |

### 4.2 程式碼格式

**✅ 正確範例**:
```python
# 每行最多 79 字元
def long_function_name(
    var_one: str,
    var_two: int,
    var_three: Optional[List[str]] = None
) -> Dict[str, Any]:
    """這是一個很長的函式簽名範例"""
    pass

# 運算子前後有空格
result = (value1 + value2) * factor
x = 1
y = 2
long_variable = 3

# 逗號後有空格
my_list = [1, 2, 3, 4]
my_dict = {'key1': 'value1', 'key2': 'value2'}

# 函式呼叫
function_call(arg1, arg2, kwarg1=value1)

# 切片不加空格
my_list[1:4]
my_list[start:end:step]
```

**❌ 錯誤範例**:
```python
# 沒有空格
result=(value1+value2)*factor

# 逗號後沒空格
my_list=[1,2,3,4]

# 不必要的空格
function_call( arg1 , arg2 , kwarg1 = value1 )
my_list[ 1 : 4 ]
```

### 4.3 空行規範

```python
# 頂層函式與類別間空兩行
def function1():
    pass


def function2():  # 空兩行
    pass


class MyClass:  # 空兩行
    """類別說明"""
    
    def method1(self):  # 類別定義與第一個方法間空一行
        pass
    
    def method2(self):  # 方法間空一行
        pass


class AnotherClass:  # 類別間空兩行
    pass
```

### 4.4 註解規範

```python
# 單行註解使用 #，# 後有一個空格
x = x + 1  # 行內註解，# 前至少兩個空格

# 區塊註解，每行開頭都有 #
# 這是一個區塊註解的範例。
# 可以有多行。
# 段落之間可用 # 分隔。
#
# 這是第二段。

# TODO: 這是待辦事項註解
# FIXME: 這是需要修正的問題
# NOTE: 這是重要說明
```

### 4.5 型別提示規範

```python
from typing import List, Dict, Optional, Union, Tuple, Any

# 基本型別
def greet(name: str) -> str:
    return f"Hello, {name}"

# 複雜型別
def process_data(
    data: List[Dict[str, Any]],
    filter_key: Optional[str] = None
) -> Tuple[int, List[Dict[str, Any]]]:
    # ...
    return count, filtered_data

# 類別屬性型別提示
class User:
    def __init__(self, name: str, age: int) -> None:
        self.name: str = name
        self.age: int = age
        self.email: Optional[str] = None
```

---

## 5. 文件撰寫規範 | Documentation Standards

### 5.1 README.md 必須包含

```markdown
# 專案名稱

簡短描述專案功能（1-2 句）

## 功能特色

- 功能 1
- 功能 2
- 功能 3

## 安裝步驟

1. 建立虛擬環境：
   ```bash
   python -m venv venv
   source venv/bin/activate  # Windows: venv\Scripts\activate
   ```

2. 安裝依賴（如有需要）：
   ```bash
   pip install -r requirements.txt
   ```

## 使用方式

```bash
python -m src.main
```

或

```bash
cd src
python main.py
```

## 專案結構

```
project/
├── src/           # 原始碼
├── data/          # 資料檔案
└── tests/         # 測試檔案
```

## 開發工具（選做）

使用 flake8 檢查程式碼風格：
```bash
flake8 src/
```

## 授權

本專案為教學用途
```

### 5.2 Docstring 規範 (Google Style)

**模組層級 Docstring**:
```python
"""模組簡短描述

詳細說明模組的功能、用途。
可以有多行。

Example:
    使用範例::
    
        from mymodule import MyClass
        obj = MyClass()
"""
```

**函式 Docstring**:
```python
def complex_function(
    param1: str,
    param2: int,
    param3: Optional[List[str]] = None
) -> Dict[str, Any]:
    """函式簡短描述（一行）
    
    詳細說明函式的功能、演算法、注意事項等。
    可以有多段說明。
    
    Args:
        param1: 參數 1 的說明
        param2: 參數 2 的說明，可以多行
            縮排對齊
        param3: 參數 3 的說明（可選參數）
    
    Returns:
        回傳值的說明。如果是複雜結構，
        說明其格式：
            {
                'key1': 'value1',
                'key2': 'value2'
            }
    
    Raises:
        ValueError: 當 param2 < 0 時
        TypeError: 當型別不符時
    
    Examples:
        >>> result = complex_function("test", 42)
        >>> print(result)
        {'status': 'success'}
    
    Note:
        重要的注意事項或使用提示
    """
    if param2 < 0:
        raise ValueError("param2 必須為正整數")
    
    return {'status': 'success'}
```

**類別 Docstring**:
```python
class TaskManager:
    """任務管理器類別
    
    負責管理所有任務的新增、刪除、查詢等操作。
    
    Attributes:
        tasks: 任務列表
        max_tasks: 最大任務數量
    
    Examples:
        >>> manager = TaskManager()
        >>> task = Task("Buy milk")
        >>> manager.add_task(task)
    """
    
    def __init__(self, max_tasks: int = 100) -> None:
        """初始化任務管理器
        
        Args:
            max_tasks: 最大任務數量，預設為 100
        """
        self.tasks: List[Task] = []
        self.max_tasks: int = max_tasks
```

---

## 6. Git 工作流程 | Git Workflow

### 6.1 初始化專案

```bash
# 1. 建立專案目錄
mkdir my_refactored_project
cd my_refactored_project

# 2. 初始化 Git
git init

# 3. 建立基本結構
mkdir src data tests
touch README.md requirements.txt .gitignore
touch src/__init__.py src/main.py

# 4. 設定 .gitignore
# (參考前面的 .gitignore 範本)

# 5. 第一次 commit
git add .
git commit -m "chore: 初始化專案結構"
```

### 6.2 Commit Message 規範

**格式**:
```
<type>: <subject>

<body>

<footer>
```

**Type 類型**:
- `feat`: 新功能
- `fix`: 修正 bug
- `refactor`: 重構（不改變功能）
- `style`: 程式碼風格調整（格式、空格等）
- `docs`: 文件更新
- `test`: 測試相關
- `chore`: 雜項（建置、設定等）
- `perf`: 效能優化

**Subject 規範**:
- 使用現在式，祈使語氣（"add" 而非 "added"）
- 不以句號結尾
- 限制在 50 字元以內

**範例**:
```bash
# 好的範例
git commit -m "feat: 新增任務優先度排序功能"
git commit -m "refactor: 拆解核心邏輯為獨立模組"
git commit -m "style: 套用 PEP 8 命名規範"
git commit -m "docs: 新增 README 安裝步驟說明"
git commit -m "fix: 修正日期格式驗證錯誤"

# 不好的範例
git commit -m "update"  # 太籠統
git commit -m "Fixed bug."  # 使用過去式
git commit -m "新增了很多功能包括任務管理、統計分析等等"  # 太長
```

### 6.3 建議的 Commit 順序

```bash
# 1. 初始化
git commit -m "chore: 初始化專案結構"

# 2. 拆解工具模組
git add src/utils.py
git commit -m "refactor: 拆解工具函式模組"

# 3. 拆解核心模組
git add src/core.py
git commit -m "refactor: 拆解核心邏輯模組"

# 4. 拆解 UI 模組
git add src/ui.py
git commit -m "refactor: 拆解使用者界面模組"

# 5. 拆解儲存模組（如需要）
git add src/storage.py
git commit -m "refactor: 拆解資料儲存模組"

# 6. 整合主程式
git add src/main.py src/__init__.py
git commit -m "refactor: 完成主程式整合"

# 7. 套用 PEP 8
git add src/
git commit -m "style: 套用 PEP 8 規範與型別提示"

# 8. 新增文件
git add README.md requirements.txt
git commit -m "docs: 新增 README 與 requirements"

# 9. 新增 docstring
git add src/
git commit -m "docs: 為所有函式新增 docstring"

# 10. 最終測試與修正
git add .
git commit -m "test: 完成功能測試與修正"
```

### 6.4 查看狀態與歷史

```bash
# 查看當前狀態
git status

# 查看提交歷史
git log
git log --oneline  # 簡潔版本
git log --graph --oneline  # 圖形化顯示

# 查看特定檔案的修改
git diff src/core.py
```

---

## 7. 完成檢核清單 | Completion Checklist

### 7.1 模組化設計 (25 分)

- [ ] 將程式拆解為至少 3 個模組
- [ ] 每個模組職責單一且明確
- [ ] 核心邏輯與 UI 分離
- [ ] 模組間透過清晰的介面溝通
- [ ] 模組間耦合度低（低依賴）
- [ ] 使用 `__init__.py` 管理套件
- [ ] 定義公開 API（`__all__`）

### 7.2 程式碼品質 (20 分)

- [ ] 所有檔案符合 PEP 8 命名規範
- [ ] 所有函式都有型別提示
- [ ] 每個函式長度 < 50 行
- [ ] 每個模組長度 < 300 行
- [ ] 移除所有 magic numbers（使用常數）
- [ ] 沒有重複的程式碼
- [ ] 適當使用私有函式/屬性（`_name`）
- [ ] 通過 flake8 檢查（無錯誤或僅少量警告）

### 7.3 文件撰寫 (15 分)

- [ ] README.md 包含專案說明
- [ ] README.md 包含安裝步驟
- [ ] README.md 包含使用方式
- [ ] README.md 包含專案結構說明
- [ ] 所有模組都有模組層級 docstring
- [ ] 所有類別都有類別 docstring
- [ ] 所有函式都有函式 docstring
- [ ] Docstring 符合 Google Style
- [ ] 複雜邏輯有註解說明
- [ ] requirements.txt 列出所有依賴

### 7.4 版本控制 (10 分)

- [ ] 初始化 Git repository
- [ ] 至少 5 個有意義的 commit
- [ ] Commit message 清楚明確
- [ ] Commit message 符合規範格式
- [ ] .gitignore 正確設定
- [ ] 不包含不必要的檔案（`__pycache__`、`.pyc` 等）

### 7.5 功能測試 (必須通過)

- [ ] 所有原始功能正常運作
- [ ] 邊界條件測試通過（空輸入、極端值）
- [ ] 錯誤處理正常運作（無效輸入）
- [ ] 檔案讀寫測試通過（如有檔案操作）
- [ ] 程式可正常啟動與結束
- [ ] 無任何 runtime error

### 7.6 專案管理 (進階)

- [ ] 建立虛擬環境
- [ ] 可透過 `python -m src.main` 執行
- [ ] 資料檔案組織清楚
- [ ] 無敏感資訊（密碼、金鑰等）
- [ ] 目錄結構清晰易懂

### 7.7 進階功能 (加分項目)

- [ ] 自訂例外類別（`exceptions.py`）
- [ ] 完整的輸入驗證
- [ ] 友善的錯誤訊息
- [ ] 撰寫單元測試
- [ ] 使用 argparse 提供 CLI 介面
- [ ] 使用 logging 模組記錄操作
- [ ] 附上程式碼分析報告（flake8/pylint）
- [ ] 效能優化與比較報告

---

## 最終檢查

在提交前，請執行以下檢查：

```bash
# 1. 檢查程式碼風格
flake8 src/

# 2. 測試程式執行
python -m src.main

# 3. 查看 Git 狀態
git status
git log --oneline

# 4. 確認檔案結構
tree .  # 或 ls -R
```

---

**祝你重構順利！**